### Assignment 1 - Dogs vs Cats

### Setup

In [1]:
%matplotlib inline

In [2]:
path = "data/dogscats/"

In [3]:
from __future__ import division, print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

import utils; reload(utils)
from utils import plots

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
Using Theano backend.


In [4]:
# batch_size=64
batch_size=8

In [5]:
import vgg16; reload(vgg16)
from vgg16 import Vgg16

In [6]:
vgg = Vgg16()

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')


#### Visualize data

In [ ]:
batches = vgg.get_batches(path+'train', batch_size=4)
imgs,labels = next(batches)

In [ ]:
print(imgs.shape)
print(labels.shape)

In [ ]:
imgs[0, :, 0, 0]

In [ ]:
plots(imgs, titles=labels)

In [ ]:
vgg.predict(imgs, True)

In [ ]:
vgg.classes[:4]

### Finetune Cats and Dogs

In [ ]:
def train_vgg():
    batch_size=64
    batches = vgg.get_batches(path+'train', batch_size=batch_size)
    val_batches = vgg.get_batches(path+'valid', batch_size=batch_size*2)
    vgg.finetune(batches)
    vgg.fit(batches, val_batches, nb_epoch=1)

    vgg.model.save_weights(weights_fullpath)

In [7]:
from keras.utils.data_utils import get_file
weights_file = 'dogscats_assignment_1.h5'
weights_fullpath = path+'models/'+weights_file
if os.path.exists(weights_fullpath):
    vgg.model.load_weights(weights_fullpath)
    print('Loaded model from cache')
else:
    train_vgg()

Loaded model from cache


### Predict results in test set

In [12]:
folderName = 'test'
num_test_images = 12500
test_batch_size = 64

# folderName = 'sample/test'
# num_test_images = 8
# test_batch_size = 1

batches = vgg.get_batches(path+folderName, batch_size=test_batch_size, shuffle=False, class_mode=None)
# with open('train.csv', 'wb') as csvfile:


Found 12500 images belonging to 1 classes.


Let's look at our predictions...

Need to convert file names to label ids

In [10]:
import re
filenames = batches.filenames

p = re.compile('.*/([0-9]+).jpg')
def find_file_id(filename):
    m = p.match(filename)
    if m is not None:
        return int(m.group(1))
    else:
        print('Could not regex filename: ', filename)
        return -1
file_ids = map(find_file_id, filenames)

Option A: Predict values using vgg test function (Not recommended. No progress)

Option B: Predict values with batches and progress

In [16]:
import math
from tqdm import tqdm

predict_file = path + 'models/predict.bc'

def predict_test():
    batches = vgg.get_batches(path+folderName, batch_size=test_batch_size, shuffle=False, class_mode=None)

    p_results = np.zeros(num_test_images)
    current_index = 0
    # Iterative loop
    for batch in tqdm(batches, total=math.ceil(num_test_images/test_batch_size)):
        if batch is None:
            break
        p = vgg.model.predict_on_batch(batch)
        p_dog = p[:, 1]
        p_size = p.shape[0]
    #     print('Predictions: {}\n Size: {}'.format(dog_p, p_size))
        new_index = current_index + p_size
        p_results[current_index:new_index] = p_dog
        current_index = new_index
        if current_index >= num_test_images:
            break
    print(p_results)
    utils.save_array(predict_file, p_results)
    return p_results

if os.path.exists(predict_file):
    p_results = utils.load_array(predict_file)
    print('Loaded predictions from cache')
else:
    p_results = predict_test()



  0%|          | 0/196.0 [00:00<?, ?it/s]

Found 12500 images belonging to 1 classes.




  1%|          | 1/196.0 [00:02<07:38,  2.35s/it]

  1%|          | 2/196.0 [00:04<07:31,  2.32s/it]

  2%|▏         | 3/196.0 [00:06<07:20,  2.28s/it]

  2%|▏         | 4/196.0 [00:09<07:15,  2.27s/it]


  3%|▎         | 5/196.0 [00:11<07:10,  2.25s/it]

  3%|▎         | 6/196.0 [00:13<07:06,  2.24s/it]

  4%|▎         | 7/196.0 [00:15<07:02,  2.24s/it]

  4%|▍         | 8/196.0 [00:17<06:58,  2.23s/it]

  5%|▍         | 9/196.0 [00:20<06:56,  2.22s/it]

  5%|▌         | 10/196.0 [00:22<06:56,  2.24s/it]

  6%|▌         | 11/196.0 [00:24<06:52,  2.23s/it]

  6%|▌         | 12/196.0 [00:26<06:50,  2.23s/it]

  7%|▋         | 13/196.0 [00:29<06:50,  2.24s/it]

  7%|▋         | 14/196.0 [00:31<06:47,  2.24s/it]

  8%|▊         | 15/196.0 [00:33<06:45,  2.24s/it]

  8%|▊         | 16/196.0 [00:35<06:44,  2.25s/it]

  9%|▊         | 17/196.0 [00:38<06:40,  2.24s/it]

  9%|▉         | 18/196.0 [00:40<06:38,  2.24s/it]

 10%|▉         | 19/196.0 [00:42<06:35,  2.23s/it]

 10%|█         | 2

[  1.4556e-10   5.9033e-01   1.3232e-07 ...,   1.6239e-10   1.4533e-10   1.0000e+00]


In [ ]:
# Verify the arrays match
print(p_results.shape)
print(len(file_ids))

In [17]:
# Clip results for better log loss
clipped_results = np.clip(p_results, 0.03, 0.97)

clipped_file = path + 'models/clip.bc'
utils.save_array(clipped_file, clipped_results)

In [ ]:
# Load saved clip file
clipped_file = path + 'models/clip.bc'
clipped_results = load_array(clipped_file)

### Combine ids with labels and save

In [19]:
import pandas as pd

agg = pd.DataFrame({'id': file_ids, 'label': clipped_results})
agg = agg.sort_values(['id'])
print(agg)
agg.to_csv(path + 'clipped.csv', index=False)


          id  label
7082       1   0.97
4067       2   0.97
3558       3   0.97
5730       4   0.97
2852       5   0.03
3896       6   0.03
9565       7   0.03
8351       8   0.03
9270       9   0.03
6412      10   0.03
2542      11   0.03
7466      12   0.97
11948     13   0.03
2109      14   0.03
9974      15   0.03
5419      16   0.03
1423      17   0.97
280       18   0.97
12187     19   0.03
8931      20   0.03
6847      21   0.97
2026      22   0.03
1513      23   0.97
1853      24   0.97
3409      25   0.03
10229     26   0.97
843       27   0.97
4989      28   0.03
10705     29   0.03
12150     30   0.97
...      ...    ...
7870   12471   0.97
11187  12472   0.97
9323   12473   0.03
6014   12474   0.03
1768   12475   0.03
503    12476   0.97
4461   12477   0.97
3674   12478   0.97
12148  12479   0.03
3069   12480   0.03
2017   12481   0.03
12428  12482   0.03
9444   12483   0.03
8808   12484   0.97
4155   12485   0.03
3222   12486   0.97
6638   12487   0.97
10322  12488   0.97


------

## Implementing VGG with Keras Backend

In [ ]:
from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers import Input
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image

In [ ]:
FILES_PATH = 'http://files.fast.ai/models/'
CLASS_FILE='imagenet_class_index.json'
fpath = get_file(CLASS_FILE, FILES_PATH+CLASS_FILE, cache_subdir='models')
with open(fpath) as f:
    class_dict = json.load(f)
# classes = [class_dict[i][1] for i in class_dict]
classes = [class_dict[str(i)][1] for i in range(len(class_dict))] # original

## Model creation

In [ ]:
def ConvBlock(layers, model, filters):
    for i in range(layers): 
        model.add(ZeroPadding2D((1,1)))
        model.add(Convolution2D(filters, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

In [ ]:
def FCBlock(model):
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

In [ ]:
# ??Convolution2D

# ??MaxPooling2D

# ??Dense

In [ ]:
# Mean of each channel as provided by VGG researchers
vgg_mean = np.array([123.68, 116.779, 103.939]).reshape((3,1,1))

def vgg_preprocess(x):
    x = x - vgg_mean     # subtract mean
    return x[:, ::-1]    # reverse axis bgr->rgb

In [ ]:
def VGG_16():
    model = Sequential()
    model.add(Lambda(vgg_preprocess, input_shape=(3,224,224)))

    ConvBlock(2, model, 64)
    ConvBlock(2, model, 128)
    ConvBlock(3, model, 256)
    ConvBlock(3, model, 512)
    ConvBlock(3, model, 512)

    model.add(Flatten())
    FCBlock(model)
    FCBlock(model)
    model.add(Dense(1000, activation='softmax'))
    return model

In [ ]:
??MaxPooling2D

??Dense

In [ ]:
model = VGG_16()

In [ ]:
fpath = get_file('vgg16.h5', FILES_PATH+'vgg16.h5', cache_subdir='models')
model.load_weights(fpath)

In [ ]:
batch_size=4

In [ ]:
def get_batches(dirname, gen=image.ImageDataGenerator(), shuffle=True, 
                batch_size=batch_size, class_mode='categorical'):
    return gen.flow_from_directory(path+dirname, target_size=(224,224), 
                class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

In [ ]:
batches = get_batches('train', batch_size=batch_size)
val_batches = get_batches('valid', batch_size=batch_size)
imgs,labels = next(batches)

# This shows the 'ground truth'
plots(imgs, titles=labels)

In [ ]:
def pred_batch(imgs):
    preds = model.predict(imgs)
    idxs = np.argmax(preds, axis=1)

    print('Shape: {}'.format(preds.shape))
    print('First 5 classes: {}'.format(classes[:5]))
    print('First 5 probabilities: {}\n'.format(preds[0, :5]))
    print('Predictions prob/class: ')
    
    for i in range(len(idxs)):
        idx = idxs[i]
        print ('  {:.4f}/{}'.format(preds[i, idx], classes[idx]))

In [ ]:
pred_batch(imgs)